In [ ]:
# Import basic extensions
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Import tensorflow imagedatagenerator
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from keras.layers import SpatialDropout2D

# Import toolboxes data and result processing
from itertools import cycle
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import regularizers

# binary

In [ ]:
# CHANGE TO 16 BATCH SIZE Input image normalization by rescaling, as 255 is maximum pixel value, the value of the image will be between 0 an 1
train_datagen = ImageDataGenerator(rescale=1/255)
val_datagen = ImageDataGenerator(rescale=1/255)

# Flow of training images
train_generator = train_datagen.flow_from_directory(
        '/Users/fannysamuelsson/Desktop/AzraProject/traintest/train',      # Input directory for the training images 
        classes = ['train_no_success','train_success'],   # Names of the directory folders, which work as labels
        target_size=(200, 200),                     # Images from the data generator will be 200 by 200 pixels
        batch_size=64,                              # Because of the limited input images
        class_mode='binary',                        # Because of the binary classification tasks on the X-ray images
        shuffle=True)            

# Flow of test images
validation_generator = val_datagen.flow_from_directory(
     #  'LUMC/NECK_preprocessed/Test_extra/',        # Input directory for the test images 
       '/Users/fannysamuelsson/Desktop/AzraProject/traintest/test',   
        classes = ['test_no_success','test_success'],   # Names of the directory folders, which work as labels
        target_size=(200, 200),                     # Images from the data generator will be 200 by 200 pixels
        batch_size=64,                              # Because of the limited input images
        class_mode='binary',                        # Because of the binary classification tasks on the X-ray images
        shuffle=False)

In [ ]:
""" # Input image normalization by rescaling, as 255 is maximum pixel value, the value of the image will be between 0 an 1
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow of training images
train_generator = train_datagen.flow_from_directory(
        'train/',      # Input directory for the training images 
        classes = ['train_no_success','train_success'],   # Names of the directory folders, which work as labels
        target_size=(200, 200),                     # Images from the data generator will be 200 by 200 pixels
        batch_size=16,                              # Because of the limited input images
        class_mode='binary',                        # Because of the binary classification tasks on the X-ray images
        shuffle=True)            

# Flow of test images
validation_generator = validation_datagen.flow_from_directory(
     #  'LUMC/NECK_preprocessed/Test_extra/',        # Input directory for the test images 
       'test/',   
        classes = ['test_no_success','test_success'],   # Names of the directory folders, which work as labels
        target_size=(200, 200),                     # Images from the data generator will be 200 by 200 pixels
        batch_size=16,                              # Because of the limited input images
        class_mode='binary',                        # Because of the binary classification tasks on the X-ray images
        shuffle=False)
 """

In [ ]:
""" from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage import exposure
import numpy as np

def enhance_spine(img):
    # Normalize to [0, 1] range
    img_normalized = (img - np.min(img)) / (np.max(img) - np.min(img))
    
    # Apply CLAHE to enhance contrast in spine region
    img_enhanced = exposure.equalize_adapthist(img_normalized, clip_limit=0.03)
    
    return img_enhanced  # Will be in [0, 1] range

# Data generators with the fixed preprocessing
train_datagen = ImageDataGenerator(
    preprocessing_function=enhance_spine,
    # No rescale needed since we normalize in the function
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=5,
    fill_mode='nearest',
    horizontal_flip=False,
    vertical_flip=False
)

validation_datagen = ImageDataGenerator(
    preprocessing_function=enhance_spine
)

train_generator = train_datagen.flow_from_directory(
        'noAugTrain/',      # Input directory for the training images 
        classes = ['train_no_success','train_success'],   # Names of the directory folders, which work as labels
        target_size=(200, 200),                     # Images from the data generator will be 200 by 200 pixels
        batch_size=10,                              # Because of the limited input images
        class_mode='binary',                        # Because of the binary classification tasks on the X-ray images
        shuffle=True)            

# Flow of test images
validation_generator = validation_datagen.flow_from_directory(
     #  'LUMC/NECK_preprocessed/Test_extra/',        # Input directory for the test images 
       'noAugTest/',   
        classes = ['test_no_success','test_success'],   # Names of the directory folders, which work as labels
        target_size=(200, 200),                     # Images from the data generator will be 200 by 200 pixels
        batch_size=28,                              # Because of the limited input images
        class_mode='binary',                        # Because of the binary classification tasks on the X-ray images
        shuffle=True)
 """

# New 4 categorical classes structure

In [ ]:
# Normalise pixel values
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen   = ImageDataGenerator(rescale=1./255)

# Training set
train_generator = train_datagen.flow_from_directory(
    '/Users/fannysamuelsson/Desktop/AzraProject/traintest4class/train',
    classes=['moderate_pain', 'no_pain', 'severe_pain', 'very_severe_pain'],  # optional fixed order
    target_size=(200, 200),
    batch_size=64,
    class_mode='categorical',   # <-- four classes → one‑hot vectors
    shuffle=True)

# Validation / test set
validation_generator = val_datagen.flow_from_directory(
    '/Users/fannysamuelsson/Desktop/AzraProject/traintest4class/test',
    classes=['moderate_pain', 'no_pain', 'severe_pain', 'very_severe_pain'],
    target_size=(200, 200),
    batch_size=64,
    class_mode='categorical',
    shuffle=False)


# optimal structure

In [ ]:
# In this part, the set-up of the actual convolutional neural network (CNN) model is described and programmed

model = tf.keras.models.Sequential([

# The first convolution layer
tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(200, 200, 3)),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.MaxPooling2D(2,2),
#tf.keras.layers.Dropout(0.1),
    
# The second convolution
#tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0, l2=1e-3)),
tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.MaxPooling2D(2,2),
#tf.keras.layers.Dropout(0.2),   
    
# The third convolution

#tf.keras.layers.Conv2D(64, (3,3), activation='relu',kernel_regularizer=regularizers.l1_l2(l1=0, l2=1e-3)),
tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Dropout(0.3),   
    
# The fourth convolution
#tf.keras.layers.Conv2D(64, (3,3), activation='relu',kernel_regularizer=regularizers.l1_l2(l1=0, l2=1e-3)),
tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.MaxPooling2D(2,2),
#tf.keras.layers.Dropout(0.4),   

# Flatten results to feed into the dense layer
tf.keras.layers.Flatten(),

# First dense layer
tf.keras.layers.Dense(512, activation='relu'),
#tf.keras.layers.BatchNormalization(),
tf.keras.layers.Dropout(0.5),

# Second dense layer
#tf.keras.layers.Dense(256, activation='relu'),
#tf.keras.layers.BatchNormalization(),
#tf.keras.layers.Dropout(0.5),

# Dense output neuron
tf.keras.layers.Dense(1, activation='sigmoid')])


# struktur 2

In [ ]:
# STRUKTUR 2 In this part, the set-up of the actual convolutional neural network (CNN) model is described and programmed

model = tf.keras.models.Sequential([

# The first convolution layer
tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(200, 200, 3)),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Dropout(0.1),
    
# The second convolution
tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0, l2=1e-3)),
#tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Dropout(0.2),   
    
# The third convolution

tf.keras.layers.Conv2D(64, (3,3), activation='relu',kernel_regularizer=regularizers.l1_l2(l1=0, l2=1e-3)),
#tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Dropout(0.3),   
    
# The fourth convolution
tf.keras.layers.Conv2D(64, (3,3), activation='relu',kernel_regularizer=regularizers.l1_l2(l1=0, l2=1e-3)),
#tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Dropout(0.4),   

# Flatten results to feed into the dense layer
tf.keras.layers.Flatten(),

# First dense layer
tf.keras.layers.Dense(512, activation='relu'),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.Dropout(0.5),

# Second dense layer
tf.keras.layers.Dense(256, activation='relu'),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.Dropout(0.5),

# Dense output neuron
tf.keras.layers.Dense(1, activation='sigmoid')])


# Categorical

In [ ]:
# CATEGORICAL In this part, the set-up of the actual convolutional neural network (CNN) model is described and programmed

model = tf.keras.models.Sequential([

# The first convolution layer
tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(200, 200, 3)),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.MaxPooling2D(2,2),
#tf.keras.layers.Dropout(0.1),
    
# The second convolution
tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0, l2=1e-3)),
#tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.MaxPooling2D(2,2),
#tf.keras.layers.Dropout(0.2),   
    
# The third convolution

tf.keras.layers.Conv2D(64, (3,3), activation='relu',kernel_regularizer=regularizers.l1_l2(l1=0, l2=1e-3)),
#tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Dropout(0.3),   
    
# The fourth convolution
tf.keras.layers.Conv2D(64, (3,3), activation='relu',kernel_regularizer=regularizers.l1_l2(l1=0, l2=1e-3)),
#tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.MaxPooling2D(2,2),
#tf.keras.layers.Dropout(0.4),   

# Flatten results to feed into the dense layer
tf.keras.layers.Flatten(),

# First dense layer
tf.keras.layers.Dense(512, activation='relu'),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.Dropout(0.5),

# Second dense layer
tf.keras.layers.Dense(256, activation='relu'),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.Dropout(0.5),

# Dense output neuron
tf.keras.layers.Dense(4, activation='softmax')])


In [ ]:
""" from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models, optimizers

# 1) load base, freeze it
base = EfficientNetB0(weights='imagenet', include_top=False, 
                      input_shape=(200,200,3))
base.trainable = False

# 2) build your head
inputs = layers.Input((200,200,3))
x = base(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(4, activation='softmax')(x)
model = models.Model(inputs, outputs)

model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
 """

In [ ]:
""" model.summary()
model.layers
model.layers[0].get_weights() """

In [ ]:
""" import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from keras.callbacks import LearningRateScheduler


base = EfficientNetB0(weights='imagenet',
                      include_top=False,
                      input_shape=(200, 200, 3))
base.trainable = False


inputs  = layers.Input((200, 200, 3))
x       = base(inputs, training=False)
x       = layers.GlobalAveragePooling2D()(x)
x       = layers.Dropout(0.2)(x)
outputs = layers.Dense(4, activation='softmax')(x)
model   = models.Model(inputs, outputs)


for layer in base.layers[-40:]:
    layer.trainable = True

labels = np.arange(len(train_generator.class_indices))
class_weight = dict(enumerate(
    compute_class_weight(class_weight='balanced',
                         classes=labels,
                         y=train_generator.classes)))
print("Class weights:", class_weight)

def step_decay_schedule(initial_lr=1e-5, decay_factor=0.9, step_size=2):
    def schedule(epoch):
        return initial_lr * (decay_factor ** np.floor(epoch / step_size))
    return LearningRateScheduler(schedule, verbose=0)

lr_sched = step_decay_schedule()

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),   # very low LR for FT
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_model.keras', monitor='val_loss', save_best_only=True
)
callbacks = [lr_sched, checkpoint]              # no EarlyStopping yet

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=callbacks,
    class_weight=class_weight,
    verbose=1
)
 """

# optimal structure

In [ ]:
""" model.compile(loss='binary_crossentropy', optimizer='sgd',  metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer=opt,  metrics=['accuracy'])

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,        # number of epochs without improvement
    restore_best_weights=True
)
checkpoint = ModelCheckpoint(
    'best_model.h5',
    monitor='val_loss',
    save_best_only=True
)

# Settings for the training-process of the CNN model
history = model.fit(train_generator,  
      epochs=60, 
      callbacks=[lr_sched],
      # early_stop,checkpoint
      verbose=1,
      validation_data = validation_generator)
      #validation_steps=25,
      #steps_per_epoch=30
 """

# loss functions and decay schedule

In [ ]:

# Loss, optimizer and metric determination
# https://www.jeremyjordan.me/nn-learning-rate/

import matplotlib.pyplot as plt
import keras.backend as K
from keras.callbacks import Callback
import numpy as np
from keras.callbacks import LearningRateScheduler
import tensorflow.keras.backend as K
import tensorflow as tf

def step_decay_schedule(initial_lr=1e-4, decay_factor=0.75, step_size=10):
 
    def schedule(epoch):
        return initial_lr * (decay_factor ** np.floor(epoch/step_size))
    
    return LearningRateScheduler(schedule)

lr_sched = step_decay_schedule(initial_lr=1e-2, decay_factor=0.9, step_size=2)

def ordinal_loss(y_true, y_pred):
    # Convert one-hot to class indices
    true_class = K.argmax(y_true, axis=-1)
    pred_class = K.argmax(y_pred, axis=-1)

    # Convert to float to compute squared error
    true_class = K.cast(true_class, tf.float32)
    pred_class = K.cast(pred_class, tf.float32)

    return K.mean(K.square(true_class - pred_class))

def combined_loss(y_true, y_pred):
    ce = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    ord_penalty = ordinal_loss(y_true, y_pred)
    return ce + 0.5 * ord_penalty



# categorical structure with combined loss functions

In [ ]:
# CATEGORICAL
model.compile(loss=combined_loss, optimizer='adam',  metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer=opt,  metrics=['accuracy'])

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,        # number of epochs without improvement
    restore_best_weights=True
)
checkpoint = ModelCheckpoint(
    'best_model.h5',
    monitor='val_loss',
    save_best_only=True
)

# Settings for the training-process of the CNN model
history = model.fit(train_generator,  
      epochs=60, 
      callbacks=[lr_sched],
      # early_stop,checkpoint
      verbose=1,
      validation_data = validation_generator)
      #validation_steps=25,
      #steps_per_epoch=30

# categorical structure with no modified loss 

In [ ]:
# CATEGORICAL
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer=opt,  metrics=['accuracy'])

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,        # number of epochs without improvement
    restore_best_weights=True
)
checkpoint = ModelCheckpoint(
    'best_model.h5',
    monitor='val_loss',
    save_best_only=True
)

# Settings for the training-process of the CNN model
history = model.fit(train_generator,  
      epochs=60, 
      callbacks=[lr_sched],
      # early_stop,checkpoint
      verbose=1,
      validation_data = validation_generator)
      #validation_steps=25,
      #steps_per_epoch=30

# binary structure

In [ ]:
# BINARY
model.compile(loss='binary_crossentropy', optimizer='adam',  metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer='sgd',  metrics=['accuracy'])

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,        # number of epochs without improvement
    restore_best_weights=True
)
checkpoint = ModelCheckpoint(
    'best_model.h5',
    monitor='val_loss',
    save_best_only=True
)

# Settings for the training-process of the CNN model
history = model.fit(train_generator,  
      epochs=60, 
      callbacks=[lr_sched],
      # early_stop,checkpoint
      verbose=1,
      validation_data = validation_generator)
      #validation_steps=25,
      #steps_per_epoch=30

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Accuracy validation
model.evaluate(train_generator)
model.evaluate(validation_generator)

# Make predictions
STEP_SIZE_TEST = validation_generator.n // validation_generator.batch_size
validation_generator.reset()
train_generator.reset()

# Get predictions
preds = model.predict(validation_generator, verbose=1)

# Convert predictions to class labels (0 or 1 for binary classification)
predicted_classes = (preds > 0.5).astype(int).flatten()

# Get true labels
true_classes = validation_generator.classes

# Create confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Print confusion matrix
print("Confusion Matrix:")
print(cm)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Success', 'Success'], 
            yticklabels=['No Success', 'Success'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Print classification report for more details
print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes, 
                          target_names=['No Success', 'Success']))


In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(true_classes, preds)

# Example find threshold with best balance
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
print(f"Optimal threshold: {optimal_threshold:.2f}")


In [ ]:
# succes no sucess
fpr, tpr, _ = roc_curve(validation_generator.classes, preds)

# Area under curve (AUC) calculation
roc_auc = auc(fpr, tpr) 

#Plotting of the ROC curve
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--', label='No Skill')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# categorical roc etc

In [ ]:
# New categorical
import math                   
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize


steps_val = math.ceil(validation_generator.samples / validation_generator.batch_size)

preds = model.predict(validation_generator, steps=steps_val)
# preds.shape  →  (N, 4)


y_true_int = validation_generator.classes          # (N,)
n_classes  = preds.shape[1]                 # 4
y_true_bin = label_binarize(y_true_int, classes=np.arange(n_classes))


fpr, tpr, roc_auc = {}, {}, {}

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], preds[:, i])
    roc_auc[i]        = auc(fpr[i], tpr[i])

fpr["micro"], tpr["micro"], _ = roc_curve(y_true_bin.ravel(), preds.ravel())
roc_auc["micro"]              = auc(fpr["micro"], tpr["micro"])


plt.figure(figsize=(8, 6))
lw = 2

class_names = list(validation_generator.class_indices.keys()) 
colors      = plt.cm.get_cmap('Set1', n_classes)

for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], lw=lw, color=colors(i),
             label=f"{class_names[i]} (AUC = {roc_auc[i]:.2f})")

# micro-average curve
plt.plot(fpr["micro"], tpr["micro"],
         color='darkorange', lw=lw+1, linestyle='--',
         label=f"micro-avg (AUC = {roc_auc['micro']:.2f})")

plt.plot([0, 1], [0, 1], color='gray', lw=lw, linestyle=':')
plt.xlim([0.0, 1.0]);  plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multi-class ROC')
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns              
from sklearn.metrics import confusion_matrix, classification_report


y_pred_int = np.argmax(preds, axis=1)   # (N,)
y_true_int = validation_generator.classes      # (N,)


cm = confusion_matrix(y_true_int, y_pred_int)
print("Confusion matrix (raw counts):\n", cm)


target_names = list(validation_generator.class_indices.keys())
print("\nClassification report:")
print(classification_report(y_true_int, y_pred_int, target_names=target_names))


plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()


In [ ]:
from collections import Counter
print("Train counts :", Counter(train_generator.classes))
print("Val   counts :", Counter(val_generator.classes))
print("Class map    :", train_generator.class_indices)

In [ ]:
print("Last epoch train acc :", history.history['accuracy'][-1])
print("Last epoch val  acc :", history.history['val_accuracy'][-1])


In [ ]:
print(train_generator.class_indices)
print(val_generator.class_indices)       

x_batch, y_batch = next(train_generator)
print("Sample batch labels:", np.argmax(y_batch, 1)[:10])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

probs = model.predict(val_generator)     # shape (N, 4)

class_map    = {v:k for k,v in val_generator.class_indices.items()}  # idx name
true_indices = val_generator.classes                                  # length N

for i in range(min(20, len(true_indices))):
    true_lbl = true_indices[i]
    pred_lbl = np.argmax(probs[i])
    print(f"{i:02d}  True: {class_map[true_lbl]:15} "
          f"Pred: {class_map[pred_lbl]:15}  Softmax: {probs[i]}")

plt.hist(np.max(probs, axis=1), bins=20)
plt.title("Confidence of predicted class")
plt.xlabel("Max soft‑max probability")
plt.ylabel("Count")
plt.show()


In [ ]:
# False positive rate and true postive rate, based on the predictions
fpr, tpr, _ = roc_curve(val_generator.classes, preds)

# Area under curve (AUC) calculation
roc_auc = auc(fpr, tpr) 

#Plotting of the ROC curve
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--', label='No Skill')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# list all data in history
print(history.history.keys())

In [ ]:
# Performance per epoch (x-axis), expressed in accuracy and loss value (y-axis)

# Model accuracy estimation
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Prediction values
preds = model.predict(validation_generator,
                      verbose=1)
#print(preds)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

s = 0; # Surgery success
u = 0; # Unsure value
n = 0; # No surgery success

for i in range(0,len(preds)):
    if preds[i]<0.50:
      #  print("Surgery success")
        s=s+1
        
    else:
     #   print("No surgery success: Person has no benefit from surgery")
        n=n+1

In [ ]:
# Prediction value evaluation

print("There are " + str(s) + " people predicted to have surgery success")
print("There are " + str(n) + " people predicted with no surgery success")
#print("There are " + str(u) + " people predicted with unsure prediction value")

# Probability evaluation 
#print("There are " + str(a) + " prediction with low probability, highly unsure")
#print("There are " + str(b) + " prediction with medium probability")
#print("There are " + str(c) + " prediction with high probability, very sure")

In [ ]:
img_path = '/Users/fannysamuelsson/Desktop/CNOC/AzraProject/test/test_success/9405515_rotated_10.png'

# GRAD CAM

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import cv2

# debug step to see the model layers
for i, layer in enumerate(model.layers):
    print(f"{i}: {layer.name} ({layer.__class__.__name__})")

# grad_cam function
def grad_cam_sequential(model, img_array, layer_name, class_idx=0):

    layer_idx = None
    for idx, layer in enumerate(model.layers):
        if layer.name == layer_name:
            layer_idx = idx
            break
    
    if layer_idx is None:
        print(f"Layer {layer_name} not found in model")
        return None, None
    
    target_layer_output = model.layers[layer_idx].output
    final_output = model.output  

    grad_model = tf.keras.models.Model(
        inputs=model.input, 
        outputs=[target_layer_output, final_output]
    )
    
  
    img_tensor = np.expand_dims(img_array, axis=0)  # shape (1, H, W, C)
    

    with tf.GradientTape() as tape:
        outputs = grad_model(img_tensor)
        

        if isinstance(outputs, list):
            conv_outputs = outputs[0]
            predictions = outputs[1]
        else:
            conv_outputs, predictions = outputs
        

        if isinstance(predictions, list):
            predictions = predictions[0]  
            

        if hasattr(predictions, 'shape') and len(predictions.shape) > 0:
            if predictions.shape[-1] == 1:

                pred_score = predictions[0][0]
            else:

                pred_score = predictions[0][class_idx]
        else:

            pred_score = predictions

    grads = tape.gradient(pred_score, conv_outputs)
    

    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    

    conv_outputs = conv_outputs[0] 
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = tf.maximum(heatmap, 0) / (tf.reduce_max(heatmap) + tf.keras.backend.epsilon())
    heatmap = heatmap.numpy()
    heatmap = cv2.resize(heatmap, (img_array.shape[1], img_array.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap_colored = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    

    original_img = np.uint8(255 * img_array)
    if len(original_img.shape) == 2:
        original_img = cv2.cvtColor(original_img, cv2.COLOR_GRAY2BGR)
    elif original_img.shape[2] == 1:
        original_img = cv2.cvtColor(original_img.squeeze(axis=2), cv2.COLOR_GRAY2BGR)
    else:
        original_img = cv2.cvtColor(original_img, cv2.COLOR_RGB2BGR)
    
    superimposed = cv2.addWeighted(original_img, 0.6, heatmap_colored, 0.4, 0)
    
    return superimposed, heatmap

def display_gradcam_sequential(model, img_path, layer_name, class_idx=0):
    """Display Grad-CAM for a sequential model"""
    try:

        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(200, 200))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = img_array / 255.0  # Normalize to [0,1]
        
        # Generate Grad-CAM
        superimposed_img, heatmap = grad_cam_sequential(model, img_array, layer_name, class_idx)
        
        if superimposed_img is None:
            print("Failed to generate Grad-CAM visualization")
            return
        
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        axes[0].imshow(img)  
        axes[0].set_title("Original Image")
        axes[0].axis("off")
        
        axes[1].imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
        axes[1].set_title("Grad-CAM Overlay")
        axes[1].axis("off")
        
        axes[2].imshow(heatmap, cmap="jet")
        axes[2].set_title("Activation Heatmap")
        axes[2].axis("off")
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Error in display_gradcam_sequential: {str(e)}")
        import traceback
        traceback.print_exc()


In [ ]:
no_success_image_path ='/Users/fannysamuelsson/Desktop/AzraProject/traintest/train/train_no_success/4081808.png'
success_image_path = '/Users/fannysamuelsson/Desktop/AzraProject/traintest/train/train_success/5005384.png'

In [ ]:
import tensorflow as tf
import numpy as np


if not isinstance(model, tf.keras.Model) or model.input is None:
    functional_in  = tf.keras.Input(shape=(200, 200, 3), name="img_in")
    functional_out = model(functional_in)          
    model          = tf.keras.Model(functional_in, functional_out, name="full_cnn")


model.build(input_shape=(None, 200, 200, 3))

img_path   = success_image_path     
class_idx  = 3                       
inner_name = "conv2d_1"              

img = tf.keras.utils.load_img(img_path, target_size=(200, 200))
img = tf.keras.utils.img_to_array(img) / 255.0
img = tf.convert_to_tensor(img[None], dtype=tf.float32)   # shape (1,200,200,3)


target_layer = model.get_layer(inner_name)   # works directly now
probe = tf.keras.Model(inputs=model.input,
                       outputs=[target_layer.output, model.output])
with tf.GradientTape() as tape:
    tape.watch(img)
    feat_maps, preds = probe(img, training=False)
    target_score     = preds[0, class_idx]          # scalar

grads = tape.gradient(target_score, feat_maps)

print(f"\n▶︎ {inner_name}")
print("   activ mean:", feat_maps.numpy().mean(),
      " std:",        feat_maps.numpy().std())
print("   grad  mean:", grads.numpy().mean(),
      " std:",        grads.numpy().std())


In [ ]:
import tensorflow as tf
import numpy as np


if not isinstance(model, tf.keras.Model) or model.input is None:
    functional_in  = tf.keras.Input(shape=(200, 200, 3), name="img_in")
    functional_out = model(functional_in)          # call Sequential as a layer
    model          = tf.keras.Model(functional_in, functional_out, name="full_cnn")


model.build(input_shape=(None, 200, 200, 3))


img_path   = success_image_path     
class_idx  = 3                       
inner_name = "conv2d_1"            


img = tf.keras.utils.load_img(img_path, target_size=(200, 200))
img = tf.keras.utils.img_to_array(img) / 255.0
img = tf.convert_to_tensor(img[None], dtype=tf.float32)   # shape (1,200,200,3)

target_layer = model.get_layer(inner_name) 

probe = tf.keras.Model(inputs=model.input,
                       outputs=[target_layer.output, model.output])

with tf.GradientTape() as tape:
    tape.watch(img)
    feat_maps, preds = probe(img, training=False)
    target_score     = preds[0, class_idx]          # scalar

grads = tape.gradient(target_score, feat_maps)

print(f"\n▶︎ {inner_name}")
print("   activ mean:", feat_maps.numpy().mean(),
      " std:",        feat_maps.numpy().std())
print("   grad  mean:", grads.numpy().mean(),
      " std:",        grads.numpy().std())


In [ ]:
from tensorflow.keras.models import Model

inputs = model.inputs if model.inputs else model.input_shape[1:]
outputs = model.outputs if model.outputs else model.layers[-1].output

new_model = Model(inputs=inputs, outputs=outputs)
print(new_model.output)

In [ ]:
dummy_input = tf.zeros((1, 200, 200, 3))
model(dummy_input)  


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

layer_name = "conv2d_1"                                    
img = image.img_to_array(image.load_img(success_image_path,
                                        target_size=(200, 200))) / 255.0

act  = tf.keras.Model(model.inputs,
                      model.get_layer(layer_name).output)(img[None]).numpy()
print("mean =", act.mean(), " std =", act.std())


In [ ]:
for i, layer in enumerate(model.layers):
    print(f"{i}: {layer.name}")
    # first conv layer
    display_gradcam_sequential(new_model, success_image_path,layer.name, class_idx=3)

    # no_success image
    display_gradcam_sequential(new_model, no_success_image_path, layer.name, class_idx=3)

In [ ]:
# Try with first conv layer
display_gradcam_sequential(new_model, success_image_path, "conv2d_3", class_idx=3)

# Try with a no_success image too
display_gradcam_sequential(new_model, no_success_image_path, "conv2d_3", class_idx=3)